# Predicción de Series Temporales con Redes Neuronales

El artículo completo con la explicación detallada en el blog: http://www.aprendemachinelearning.com/

## Usaremos Keras y Tensorflow

Importamos las Librerías que vamos a utilizar

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('fast')

from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from sklearn.preprocessing import MinMaxScaler

### Cargamos nuestro Dataset

In [ ]:
df = pd.read_csv('Residuales.csv')
df['Periodo'] = pd.to_datetime(df['Periodo'])
df.set_index('Periodo', inplace =True)
df

,Residuales
Periodo,
2001-01-01,204.202959
2001-02-01,-12926.864960
2001-03-01,5941.806221
2001-04-01,-4052.989250
2001-05-01,9629.944185
...,...
2023-02-01,-10990.455430
2023-03-01,1558.534880
2023-04-01,15761.494470


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 270 entries, 2001-01-01 to 2023-06-01
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Residuales  270 non-null    float64
dtypes: float64(1)
memory usage: 4.2 KB


### Cargamos las predicciones

In [ ]:
Predicciones = pd.read_csv("predicciones.csv",sep=',')
Predicciones = np.ravel(Predicciones)
Predicciones

array([256054.74987657, 248318.38162202, 256271.79253835, 251683.1960663 ,
       263829.48262881, 263584.21993736, 265155.4302604 , 271190.24132205,
       263163.95801613, 263007.81534794, 261623.65898023, 273539.56349901,
       241912.59078922, 234061.34840481, 238430.76688637, 233893.44918756,
       246742.90997693, 246556.79677497])

### Cargamos los datos de testeo

In [ ]:
Serie_plata = pd.read_csv("CP2_more_PERI_2021.csv",sep=',')
Serie_plata = np.ravel(Serie_plata[(270-18):]['CP2_more_PERI_2021'])
Serie_plata

array([254325.67666696, 242848.52301537, 241446.119808  , 247226.5134573 ,
       251878.22896203, 257645.018077  , 249824.96724563, 270285.83457504,
       254721.599202  , 263906.42397945, 244462.15808061, 276693.188497  ,
       205489.0143848 , 223070.89297369, 239989.30176629, 249654.94365682,
       242527.282439  , 274253.991975  ])

# Preprocesado de los datos

###### Definimos una función para que adecue los datos de entrada en una red neuronal

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

##### Creamos una lista

In [ ]:
Lista  = pd.DataFrame({
    'Corte': [],
    'Mape': [],
    'Mae':[],
    'RMSE': []
})
Lista

,Corte,Mape,Mae,RMSE


##### Este bucle nos devuelve, para varios cortes(nodos de entrada), los diferentes valores en MAE, MAPE Y RMSE.

In [ ]:
for i in range(3,50):
  def crear_modeloFF():
    model = Sequential()
    model.add(Dense(i, input_shape=(1,i),activation='linear'))
    model.add(Flatten())
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_absolute_error',optimizer='Adam',metrics=["mse"])
    model.summary()
    return model
  # load dataset
  values = df.values
  # ensure all data is float
  values = values.astype('float32')
  # normalize features
  scaler = MinMaxScaler(feature_range=(-1, 1))
  values=values.reshape(-1, 1) # esto lo hacemos porque tenemos 1 sola dimension
  scaled = scaler.fit_transform(values)
  # frame as supervised learning
  reframed = series_to_supervised(scaled, i, 1)

  values = reframed.values
  n_train_days = 270 - (18+i)
  train = values[:n_train_days, :]
  test = values[n_train_days:, :]
  # split into input and outputs
  x_train, y_train = train[:, :-1], train[:, -1]
  x_val, y_val = test[:, :-1], test[:, -1]
  # reshape input to be 3D [samples, timesteps, features]
  x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
  x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
  print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)
  EPOCHS=40
  model = crear_modeloFF()
  history=model.fit(x_train,y_train,epochs=EPOCHS,validation_data=(x_val,y_val),batch_size=i)
  results=model.predict(x_val)
  compara = pd.DataFrame(np.array([y_val, [x[0] for x in results]])).transpose()
  compara.columns = ['real', 'prediccion']

  inverted = scaler.inverse_transform(compara.values)

  compara2 = pd.DataFrame(inverted)
  compara2.columns = ['real', 'prediccion']
  compara2['diferencia'] = compara2['real'] - compara2['prediccion']

  y_pred_test = np.array(compara2['prediccion'])
  Serie_plata_predic = y_pred_test + Predicciones
  Serie_plata_predic

  MAPE = np.mean(np.abs((Serie_plata-Serie_plata_predic)/(Serie_plata)*100))
  MAE = mean_absolute_error(Serie_plata, Serie_plata_predic)
  RMSE = sqrt(mean_squared_error(Serie_plata, Serie_plata_predic))
  nueva_fila = {'Corte': i, 'Mape': MAPE, 'Mae':MAE,'RMSE': RMSE}
  Lista = Lista.append(nueva_fila, ignore_index=True)

####### De todos los cortes posibles, se obtiene el que tiene menor MAPE.

In [ ]:
Lista.loc[Lista['Mape'].idxmin()]

Corte        3.000000
Mape         4.101647
Mae       9877.024290
RMSE     13163.489109
Name: 0, dtype: float64